# 0. Setup Paths

In [ ]:
import os

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet_combined_final'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [ ]:
paths

# 1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

In [ ]:
# https://www.tensorflow.org/install/source_windows
os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Install Tensorflow Object Detection 

if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 


In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
#!pip install tensorflow --upgrade

In [ ]:
!pip uninstall protobuf matplotlib -y
!pip install protobuf matplotlib==3.2

In [ ]:
!pip install Pillow

In [ ]:
!pip install pyyaml

In [ ]:
import object_detection

In [ ]:
!pip list

In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

# 2. Check Annotation Files and Create Label Map

In [ ]:
##Check image rotation
from PIL import Image, ExifTags

# Run to format annotation files correctly
path_extension = "test"
file_names = os.listdir(os.path.join(paths['IMAGE_PATH'], path_extension))
count = 0
found = 0
t = 0
for f in file_names:
    if f[-1] != "l":
        image = Image.open(os.path.join(paths['IMAGE_PATH'], path_extension, f))
        try:
            if dict(image._getexif().items()).get(274) == 6:
                print("bad", f)
        except:
            print(end="")
        image.close()

In [ ]:
# Run to remove occluded line from annotation file
path_extension = "test"
file_names = os.listdir(os.path.join(paths['IMAGE_PATH'], path_extension))
count = 0
num_found = 0
t = 0
for f in file_names:
    if f[-1] == "l":
        with open(os.path.join(paths['IMAGE_PATH'], path_extension, f), "r") as file:
            lines = file.readlines()
            found = False
            for line in lines:
                if line.strip("\n") == "		<occluded>0</occluded>":
                    found = True
                    num_found += 1
        if found == True:
            with open(os.path.join(paths['IMAGE_PATH'], path_extension, f), "w") as write_file:
                for line in lines:
                    if line.strip("\n") != "		<occluded>0</occluded>":
                        write_file.write(line)
        count += 1
    t += 1

print("Annotations checked: ", count)
print("Number found wrong: ", num_found)

In [ ]:
#make sure ymin and xmax are in the right order
file_names = os.listdir(os.path.join(paths['IMAGE_PATH'], path_extension))
count = 0
found = 0
t = 0
for f in file_names:
    if f[-1] == "l":
        with open(os.path.join(paths['IMAGE_PATH'], path_extension, f), "r") as file:
            lines = file.readlines()
        with open(os.path.join(paths['IMAGE_PATH'], path_extension, f), "w") as write_file:
            ymin_found = False
            xmax_found = False
            xmax_line = ""
            for line in lines:
                if line.strip("\n")[4:8] != "xmax" and line.strip("\n")[4:8] != "ymin" and line.strip("\n")[4:8] != "ymax":
                    write_file.write(line)
                elif line.strip("\n")[4:8] == "xmax":
                    if ymin_found == True:
                        write_file.write(line)
                    else:
                        xmax_line = line
                        xmax_found = True
                elif line.strip("\n")[4:8] == "ymin":
                    if xmax_found == False:
                        ymin_found = True
                        write_file.write(line)
                    else:
                        write_file.write(line)
                        write_file.write(xmax_line)
                elif line.strip("\n")[4:8] == "ymax":
                    write_file.write(line)
                    ymin_found = False
                    xmax_found = False
                    xmax_line = ""
        count += 1
    t += 1



In [ ]:
#make sure all boxes are positive
path_extension = "train"
file_names = os.listdir(os.path.join(paths['IMAGE_PATH'], path_extension))
count = 0
checked = 0
found = 0
heights = 0
widths = 0
overs = 0
t = 0
areas_check = 0
for f in file_names:
    if f[-1] == "l":
        with open(os.path.join(paths['IMAGE_PATH'], path_extension, f), "r") as file:
            lines = file.readlines()
            height = 0
            width = 0
            xmin = 0
            ymin = 0
            xmax = 0
            ymax = 0
            ymax_checked = False
            for line in lines:
                if line.strip("\n")[3] == "w":
                    stuff = line.split(">")
                    number = stuff[1].split("<")[0]
                    widths += 1
                    width = int(number)
                    if width < 0:
                        print("negative width")
                if line.strip("\n")[3] == "h":
                    stuff = line.split(">")
                    number = stuff[1].split("<")[0]
                    heights += 1
                    height = int(number)
                    if height < 0:
                        print("negative height")
                if line.strip("\n")[4] == "x" or line.strip("\n")[4] == "y":
                    stuff = line.split(">")
                    number = stuff[1].split("<")[0]
                    checked += 1
                    if int(number) < 0:
                        found += 1
                    if line.strip("\n")[4] == "x":
                        if int(number) > width:
                            over += 1
                    if line.strip("\n")[4] == "y":
                        if int(number) > height:
                            over += 1
                    if line.strip("\n")[4:8] == "xmin":
                        xmin = int(number)
                    if line.strip("\n")[4:8] == "ymin":
                        ymin = int(number)
                    if line.strip("\n")[4:8] == "xmax":
                        xmax = int(number)
                    if line.strip("\n")[4:8] == "ymax":
                        if ymax_checked == True:
                            #print(f)
                            print(end="")
                        ymax_checked = True
                        areas_check += 1
                        ymax = int(number)
                        '''
                        if xmin == 0 or xmax == 0 or ymin == 0:
                            print("All zero")
                        if (xmin > xmax or ymin > ymax):
                            print("Wrong order!")
                        if (xmax - xmin) * (ymax - ymin) <= 0:
                            print("zero area")
                        if width == xmax:
                            print("same width", f)
                        if height == ymax:
                            print("same height", f)
                        '''
        count += 1
        #print(width, height, xmin, ymin, xmax, ymax, f)
    t += 1
print("Files checked:", count)
#print(checked)
print("Found incorret:",found)
#print(heights)
#print(widths)
#print(overs)
print("Boxes checked:", areas_check)

In [ ]:
#Create label map
class_names = ["cat-Abyssinian", "cat-Bengal", "cat-Birman", "cat-Bombay", "cat-British_Shorthair", "cat-Egyptian_Mau", 
               "cat-Maine_Coon", "cat-Persian", "cat-Ragdoll", "cat-Russian_Blue", "cat-Siamese", "cat-Sphynx", 
               "dog-american_bulldog", "dog-american_pit_bull_terrier", "dog-basset_hound", "dog-beagle", "dog-boxer", 
               "dog-chihuahua", "dog-english_cocker_spaniel", "dog-english_setter", "dog-german_shorthaired", 
               "dog-great_pyrenees", "dog-havanese", "dog-japanese_chin", "dog-keeshond", "dog-leonberger", 
               "dog-miniature_pinscher", "dog-newfoundland", "dog-pomeranian", "dog-pug", "dog-saint_bernard", "dog-samoyed", 
               "dog-scottish_terrier", "dog-shiba_inu", "dog-staffordshire_bull_terrier", "dog-wheaten_terrier", 
               "dog-yorkshire_terrier", "bella"]
labels = []
count = 1
for c in class_names:
    new_lab = {}
    new_lab["name"] = c
    new_lab["id"] = count
    count += 1
    
    labels.append(new_lab)
#print(labels)
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Create TF records

In [ ]:
# OPTIONAL IF RUNNING ON COLAB
ARCHIVE_FILES = os.path.join(paths['IMAGE_PATH'], 'archive.tar.gz')
if os.path.exists(ARCHIVE_FILES):
  !tar -zxvf {ARCHIVE_FILES}

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [ ]:
files['TF_RECORD_SCRIPT']
paths['ANNOTATION_PATH']

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

In [ ]:
!pip install pytz

# 4. Copy Model Config to Training Folder

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

# 5. Update Config For Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 8
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=20000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [ ]:
print(command)

In [ ]:
#!{command}

# 7. Evaluate the Model

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [ ]:
print(command)

In [ ]:
!{command}

# 8. Load Train Model From Checkpoint

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-75')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 9. Detect from all Test Images

In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import pandas as pd

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
images = os.listdir(os.path.join(paths['IMAGE_PATH'], "test"))

In [ ]:
import time
import random

random_images = []
for x in images:
    if x[-1] != "l":
        random_images.append(x)

longest = 0
final_data = []

save_folder = ""

for i_path in random_images:
    curr_image_data = []
    curr_image_data.append(i_path)
    
    IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', i_path)
    
    img = cv2.imread(IMAGE_PATH)
    width, height, colors = img.shape
    curr_image_data.append(width)
    curr_image_data.append(height)
    
    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

    start = time.time()
    detections = detect_fn(input_tensor)
    finish = time.time()

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections
    
    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
    boxes = np.squeeze(detections["detection_boxes"])
    classes = np.squeeze(detections["detection_classes"])
    scores = np.squeeze(detections["detection_scores"])
    count = num_detections
    
    res = []
    for i in range(count):
        if scores[i] >= -1:
            #print("Score: ", scores[i])
            result = {
                'bounding_box': boxes[i],
                'class_id': classes[i],
                'score': scores[i]
            }
            res.append(result)
    
    res_counter = 0
    for result in res:
        curr_image_data.append(result)
        res_counter += 1
        ymin, xmin, ymax, xmax = result['bounding_box']
        xmin = int(max(1,xmin * width))
        xmax = int(min(width, xmax * width))
        ymin = int(max(1, ymin * height))
        ymax = int(min(height, ymax * height))
    
    if len(curr_image_data) > longest:
        longest = len(curr_image_data)
    final_data.append(curr_image_data)
    
    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.6,
                agnostic_mode=False)
    
    final_image = cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)
    plt.imshow(final_image)
    if save_folder != "":
        save_place = save_folder + i_path
        cv2.imwrite(save_place, final_image)
    plt.show()

    print("FPS:", 1 / (finish - start))

possible_columns = ['id', 'width', 'height', '1box', '2box', '3box', '4box', '5box', '6box', '7box', '8box', '9box', '10box', '11box', '12box', '13box', '14box', '15box', '16box', '17box', '18box', '19box', '20box', '21box', '22box', '23box', '24box', '25box', '26box', '27box', '28box', '29box', '30box', '31box', '32box', '33box', '34box', '35box', '36box', '37box', '38box', '39box', '40box', '41box', '42box', '43box', '44box', '45box', '46box', '47box', '48box', '49box', '50box', '51box', '52box', '53box', '54box', '55box', '56box', '57box', '58box', '59box', '60box', '61box', '62box', '63box', '64box', '65box', '66box', '67box', '68box', '69box', '70box', '71box', '72box', '73box', '74box', '75box', '76box', '77box', '78box', '79box', '80box', '81box', '82box', '83box', '84box', '85box', '86box', '87box', '88box', '89box', '90box', '91box', '92box', '93box', '94box', '95box', '96box', '97box', '98box', '99box', '100box', '101box', '102box', '103box', '104box', '105box', '106box', '107box', '108box', '109box', '110box', '111box', '112box', '113box', '114box', '115box', '116box', '117box', '118box', '119box', '120box', '121box', '122box', '123box', '124box', '125box', '126box', '127box', '128box', '129box', '130box', '131box', '132box', '133box', '134box', '135box', '136box', '137box', '138box', '139box', '140box', '141box', '142box', '143box', '144box', '145box', '146box', '147box', '148box', '149box', '150box']
df = pd.DataFrame(final_data, columns=possible_columns[0:longest])
df.to_csv("tensorflow_combined_results.csv")

# 10. Real Time Detections from your Webcam

In [ ]:
!pip uninstall opencv-python-headless -y

In [ ]:
from IPython.display import clear_output
import time

cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    
    clear_output()
    start = time.time()
    detections = detect_fn(input_tensor)
    finish = time.time()
    print("FPS:", 1 / (finish - start))
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

# 10. Freezing the Graph

In [ ]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [ ]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

In [ ]:
print(command)

In [ ]:
!{command}

# 11. Making a prediction video

In [2]:
import cv2
import numpy as np
import time

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture("test.mp4")

# Check if camera opened successfully
if (cap.isOpened()== False):
    print("Error opening video stream or file")

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
print("")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
out = cv2.VideoWriter('Tensorflow.mp4', fourcc, 10, (frame_width,frame_height))
# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        image_np = np.array(frame)
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        
        start = time.time()
        detections = detect_fn(input_tensor)
        finish = time.time()

        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        boxes = np.squeeze(detections["detection_boxes"])
        classes = np.squeeze(detections["detection_classes"])
        scores = np.squeeze(detections["detection_scores"])
        count = num_detections
        
        label_id_offset = 1
        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.6,
                agnostic_mode=False)
        #final_image = cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)
        
        out.write(image_np_with_detections)
        
        # Display the resulting frame
        cv2.imshow('Frame',frame)
        
        print("FPS:", 1 / (finish - start))

        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    # Break the loop
    else:
        break

cap.release()
out.release()

# Closes all the frames
cv2.destroyAllWindows()


NameError: name 'tf' is not defined